## 🔮 Deep Learning in Practice


As we have seen, Keras forms the core of the TensorFlow deep learning pipeline and we will see later that it can be viewed as a standalone package that doesn't necessarily depend on TensorFlow.

<img src="https://i.imgur.com/SbcHrMK.png">

### 📊 Keras.utils

`Keras.utils` has data loading capabilities (including different formats: image, audio, text, time series) as well as other utilities (e.g., plotting model architecture). Recall, `torch.utils.data` is also what handled data loading in PyTorch.

In [1]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory


train_data = image_dataset_from_directory(
    directory='./data/Hymenoptera/train',
    labels='inferred',                              # i.e., infer classes from folders (otherwise, directory structure ignored)
    label_mode='int',                               # automatically assign labels as integers (can also choose one-hot)
    batch_size=32,                                  # batch size
    image_size=(224, 224),                          # Size to resize images to after they are read from disk
    seed=42  
)

val_data = image_dataset_from_directory(
    directory='./data/Hymenoptera/val',
    batch_size=32,  
    image_size=(224, 224), 
    seed=42 
)

# this is equivalent to doing lots of tensorflow work:
assert isinstance(train_data, tf.data.Dataset)

Found 244 files belonging to 2 classes.
Found 153 files belonging to 2 classes.


Similarily, there is `text_dataset_from_directory` function and `audio_dataset_from_directory` where `labels='inferred'` has the same behaviour. Check them out [here](https://keras.io/api/data_loading/). 

PyTorch also has the capability of easily reading different formats but they as delegated to friendly packages such as`torchvision`, `torchaudio` and `torchtext`.

### 🗂️ Keras.datasets

Likewise, `Keras.datasets` has some built-in datasets for image, text and regression. In PyTorch, image datasets were in `torchvision` and text datasets were in `torchtext` (as well as audio datasets in `torchaudio`). 

For a list of datasets, [see](https://keras.io/api/datasets/).

Note that it reads them as Numpy arrays.

In [2]:
from tensorflow.keras.datasets import fashion_mnist
import numpy as np

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
assert x_train.shape == (60000, 28, 28)
assert type(x_train) == np.ndarray

This module is limited in terms of the number of datasets available and that it returns them in Numpy arrays (still need to wrap in TensorFlow dataset). Much more datasets can be loaded in `Dataset` format from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview#all_datasets).

In [8]:
# !pip install tensorflow-datasets
import tensorflow_datasets as tfds

# Construct a tf.data.Dataset
train_dataset = tfds.load('mnist', split='train', as_supervised=True, shuffle_files=True)

# Build your input pipeline
train_dataset = train_dataset.shuffle(1000).batch(128).take(5)
for xb, yb in train_dataset:
  assert xb.shape == (128, 28, 28, 1)

2024-05-18 13:49:46.989541: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


### 🏷️ Keras.layers

Like what you would expect, this implements a large variety of deep learning layers (e.g., linear, convolutional, recurrent, attention, etc.) and activation functions.

They behave much like in PyTorch:

In [19]:
import tensorflow as tf
from tensorflow.keras import layers

layer = layers.Dense(32, activation='relu')         # a layer with 32 neurons; input size will be inferred on first call
inputs = tf.random.uniform(shape=(16, 20))          # batch of 16 examples, each of dimensionality 20
outputs = layer(inputs)
outputs.shape

TensorShape([16, 32])

Defining custom layers similar to PyTorch is also possible:

In [21]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, input_dim=20, output_dim=32):
        super().__init__()
        self.w = self.add_weight(shape=(input_dim, output_dim),trainable=True)
        self.b = self.add_weight(shape=(output_dim,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    

layer = Linear(20, 32)                               # a layer with 32 neurons; input size will be inferred
inputs = tf.random.uniform(shape=(16, 20))          # batch of 16 examples, each of dimensionality 20
outputs = layer(inputs)
outputs.shape

TensorShape([16, 32])

It's straightforward make your layer infer the `input_dim` via the `build` function and it's considered good practice. Check [here](https://keras.io/guides/making_new_layers_and_models_via_subclassing/) to learn how. 

For now, let's make another block using the layer we just created and use activation functions along the way:

In [26]:
class MLPBlock(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        self.linear_1 = Linear(20, 32)
        self.linear_2 = Linear(32, 50)
        self.linear_3 = Linear(50, 1)
        self.relu = tf.keras.layers.ReLU()          # can be treated as a layer like PyTorch (at least in this case)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = self.relu(x)
        x = self.linear_2(x)
        x = tf.keras.activations.relu(x)            # a THIRD alternative (similar to torch.nn.functional) => use this or specify in dense
        return self.linear_3(x)
    

mlp = MLPBlock()
y = mlp(tf.random.uniform(shape=(16, 20)) )  
# Notice that we get automatic parameter tracking similar to torch.nn.Module:
print(f"the block has a total of {mlp.count_params()} parameters distribued over {len(mlp.weights)} weights")

the block has a total of 2373 parameters distribued over 6 weights


### 📉 Keras.losses

Again, an API too similar to that of PyTorch:

In [32]:
y_true = [0, 1, 0, 0]
y_pred = [0.01, 0.9, 0.01, 0.03]
bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
bce(y_true, y_pred).numpy()                 # NOTE: PyTorch did bce(y_pred, y_pred)

0.03897997

In [36]:
mse = tf.keras.losses.mean_squared_error(y_true, y_pred).numpy()        # snake_case for functional loss
mse

0.002775001

### 🚀 Keras.optimizers

Similar to PyTorch except that the parameters and their gradients must be passed with each optimization step and `apply_gradients` instead of `step`.

In [48]:
# Instantiate the model
mlp = MLPBlock()

# Define optimizer and loss
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
criterion = tf.keras.losses.BinaryCrossentropy(from_logits=False)

# Generate random input data
x = tf.random.uniform(shape=(16, 20))
y_true = tf.random.uniform(shape=(16, 1))

# Take one step of optimization
with tf.GradientTape() as tape:
    y_pred = mlp(x)
    loss = criterion(y_true, y_pred)

gradients = tape.gradient(loss, mlp.trainable_variables)                #  mlp.trainable_variables like mlp.params in PyTorch
optimizer.apply_gradients(zip(gradients, mlp.trainable_variables))      # it takes a list of gradient, variable pairs (hence we used zip)

print("Loss after one step:", loss.numpy())

Loss after one step: 0.77470434


Can also pass a scheduler while instantiating the optimizer:

In [49]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

### 📈 Keras.metrics

PyTorch does not natively support metrics. There is an independent package [torch metrics](https://github.com/Lightning-AI/torchmetrics) for that but native support as in Keras is even better. They are treated similar to loss functions but also have the ability to be updated over multiple steps (e.g., batches):

In [46]:
# Define metric
accuracy = tf.keras.metrics.Accuracy()

# Generate some example data
y_true = tf.constant([1, 0, 1, 1])  
y_pred = tf.constant([0, 0, 1, 1])  

# Update state with the first half of the data
accuracy.update_state(y_true[:2], y_pred[:2])
print("Accuracy:", accuracy.result().numpy())

# Calculate Accuracy after also including the second half
accuracy.update_state(y_true[2:], y_pred[2:])
print("Accuracy:", accuracy.result().numpy())

Accuracy: 0.5
Accuracy: 0.75


As can be [seen](https://keras.io/api/metrics/), they cover most metrics you will ever need.

<img src="https://i.imgur.com/7cwS47a.png">

At this point we only need to look at `keras.callbacks`, `keras.ops`. But before we do we will talk about two more keras packages: `KerasCV`, `KerasNLP`:

### 👁️ KerasCV

- Similar to `torchvision`, this provides transformations for images (but in the form of layers this time)
- Like `torchvision` it also provides pretrained vision models (another submodule `keras.applications` also comes with pretrained vision models)
- Unlike `torchvision`, no datasets or data reading functions (we discussed both of these above) but provides more loss functions for vision

Let's use to load a StableDiffusion model (out of scope) that can convert text to images by repeately applying diffusion on noise:

Many of the transformations can be done in a more low-level fasion via `tensorflow.image`.

In [74]:
import keras_cv

# Load architecture and weights from preset
backbone = keras_cv.models.MobileNetV3Backbone.from_preset("mobilenet_v3_large_imagenet", load_weights=False)

input_data_shape = (8, 224, 224, 3)          # batch of 8 images each of dimensions (224, 224, 3) => notice difference to PyTorch (channels)
input_data = tf.random.uniform(shape=input_data_shape, minval=0, maxval=1)
output = backbone(input_data)
output.shape                                 # the backbone does not include the output classification layer

TensorShape([8, 7, 7, 960])

Let's see an example augmentation:

In [75]:
random_hue = keras_cv.layers.RandomHue(factor=0.3, value_range=[0,1])
augmented_images = random_hue(input_data)
assert not tf.reduce_all(tf.equal(augmented_images, input_data))

Note that other augmentations also exist as layers in `keras.layers`.

### 📝 KerasNLP

Parallel to `KerasCV`, we also have `KerasNLP` for text. It provides:
- Pretrained text models
- Preprocessing layers (e.g., to add start/end token, random deletion of tokens, etc.)

As well as, tokenizers, transformer layers and NLP metrics

In [ ]:
import keras_nlp

vocab = ["[UNK]", "the", "qu", "##ick", "br", "##own", "fox", "."]
inputs = ["The quick brown fox."]
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
     vocabulary=vocab,
     sequence_length=10,
     lowercase=True,
 )
outputs = tokenizer(inputs)
np.array(outputs)

They have many common NLP models (backbone, full classlifier, preprocessor layers, tokenizers) as well.

In [63]:
input_data = {
    "token_ids": np.ones(shape=(1, 12), dtype="int32"),                     # example sequence
    "segment_ids": np.array([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0]]),        # to distinguish two sequences
    "padding_mask": np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]]),       # to ignore padding tokens
}

# Pretrained BERT encoder.
model = keras_nlp.models.BertBackbone.from_preset("bert_base_en_uncased", load_weights=False)
model(input_data)['sequence_output'].shape          # maps the 12 tokens into contextual embeddings of 768D

TensorShape([1, 12, 768])

However, [HuggingFace](https://huggingface.co/models) is more common for such types of workflow.